# array 方法介绍

## 同时出现在两个数组中的元素
## 第二个数组比第一个数组多了多少个元素

In [33]:
# 提高输出效率库
from IPython.core.interactiveshell import InteractiveShell # 实现 notebook 的多行输出
InteractiveShell.ast_node_interactivity = 'all' #默认为'last'

In [18]:
import numpy as np

# 示例数据，两个月的用户ID：当月和下月
current_month = np.array([1, 2, 2, 3, 4, 5, 7])
next_month = np.array([2, 3, 4, 5, 6, 6, 6])

# 获取在这个月消费后下个月仍然消费的用户ID
# np.intersect1d()函数可以计算交集的长度，而且会自动去重
common_users = np.intersect1d(current_month, next_month)

print(f'该月用户消费记录：{current_month}')
print(f'下月用户消费记录：{next_month}')

# 输出结果：在这个月消费后下个月仍然消费的用户数量
print(f'有 {len(common_users)} 个用户在该月消费后下个月依然消费。id分别为：{common_users}')

该月用户消费记录：[1 2 2 3 4 5 7]
下月用户消费记录：[2 3 4 5 6 6 6]
有 4 个用户在该月消费后下个月依然消费。id分别为：[2 3 4 5]


In [20]:
print(f'该月用户消费记录：{current_month}')
print(f'下月用户消费记录：{next_month}')
# 获取第二个月新增的用户ID
new_users = np.setdiff1d(next_month, current_month)
# 输出结果：第二个月新增的用户数量
print(f'下个月中，有 {len(new_users)} 个新增用户，id分别为：{new_users}')

该月用户消费记录：[1 2 2 3 4 5 7]
下月用户消费记录：[2 3 4 5 6 6 6]
下个月中，有 1 个新增用户，id分别为：[6]


# 数据读入与处理

In [22]:
import pandas as pd

df = pd.read_csv('customers_data.csv')
df['付款时间'] = pd.to_datetime(df['付款时间'])
df['付款年月'] = df['付款时间'].dt.strftime('%Y-%m')
df.sample(10)

,脱敏客户ID,付款时间,支付金额,付款年月
39799,cumid30822,2024-02-29 02:21:20,76.39,2024-02
5729,cumid5274,2023-10-17 16:27:55,17.98,2023-10
1109,cumid1061,2023-09-17 13:10:32,25.33,2023-09
24754,cumid2850,2024-01-06 13:59:01,9.64,2024-01
32405,cumid25457,2024-02-06 17:03:52,44.13,2024-02
22293,cumid18324,2023-12-28 15:52:29,116.91,2023-12
27044,cumid13784,2024-01-12 22:44:28,0.00,2024-01
865,cumid828,2023-09-13 17:16:02,19.35,2023-09
18962,cumid15867,2023-12-12 11:56:46,36.32,2023-12
6194,cumid5694,2023-10-18 11:51:20,350.02,2023-10


In [23]:
df.sample(10).to_clipboard()

# 单月新增的留存情况
以 2023年 9 月为例，计算之后的留存情况

In [21]:
# 2023年9月用户新增情况
Nov_new = df.query('付款年月 == "2023-09"')
print(f'2023-09 消费记录数：{len(Nov_new)}，新增用户数（唯一ID）：{Nov_new["脱敏客户ID"].nunique()}')

# 9 月新增的用户中，有多少留存到了 10 月
# 与历史数据做匹配，即客户昵称在 2023-10 月且也在 9 月的
month = '2023-10'
month_customer = df[df['付款年月'] == month]
common_users = np.intersect1d(Nov_new["脱敏客户ID"], month_customer["脱敏客户ID"])
print(f'{month} 的用户中，有 {len(common_users)} 个是上个月留存下来的')

2023-09 消费记录数：2187，新增用户数（唯一ID）：2031
2023-10 的用户中，有 252 个是上个月留存下来的


In [25]:
# 循环构造
print('2023-09 的客户在后续月份中的留存情况...')
stay = []
for i in ['2023-10', '2023-11', '2023-12', '2024-01', '2024-02']:
    next_month = df[df['付款年月'] == i]
    # 2023-9 的客户还出现在时间 i 的 DataFrame 中
    common_users = np.intersect1d(Nov_new["脱敏客户ID"], next_month["脱敏客户ID"])
    stay.append( [i+'留存人数：', len(common_users)] )
stay

2023-09 的客户在后续月份中的留存情况...


[['2023-10留存人数：', 252],
 ['2023-11留存人数：', 216],
 ['2023-12留存人数：', 163],
 ['2024-01留存人数：', 156],
 ['2024-02留存人数：', 164]]

# 循环构造

In [17]:
month_list = df['付款年月'].unique()
month_list

array(['2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02'],
      dtype=object)

## 大循环外部

In [27]:
for i in range(0, len(month_list)-1):
                   # len()-1 的原因：最后一个月之后就没有数据了
    # 筛选出 month_list 中的每月消费，并统计客户数量
    print(f'下面统计: {month_list[i]} 的新增情况...')
    current_data = df[ df['付款年月']==month_list[i] ]
    current_clients = current_data['脱敏客户ID'].unique()
    
    # =========================== 统计新增情况 ==================================
    # 跳过数据集中的第一个月，因为没有历史数据来验证该客户是否为新增客户
    if i == 0:
        print(f'{month_list[i]} 是第一个月，无需验证客户是否为新增客户。')
        new_clients_num = len(current_clients)
        print(f'该月的新增用户数为：{new_clients_num}')
    else:
        # 筛选该月（current_month）之前的所有历史消费记录
        history_month = month_list[:i]
        print(f'{month_list[i]} 的历史年月为：{history_month}')
        history_data = df[ df['付款年月'].isin(history_month) ]
        # 筛选未在历史消费记录中出现过的新增客户
        new_users = np.setdiff1d(current_data['脱敏客户ID'], history_data['脱敏客户ID'])
        print(f'相较于历史年月，该月的新增客户数为：{len(new_users)}')
    print('\n')

下面统计: 2023-09 的新增情况...
2023-09 是第一个月，无需验证客户是否为新增客户。
该月的新增用户数为：2031


下面统计: 2023-10 的新增情况...
2023-10 的历史年月为：['2023-09']
相较于历史年月，该月的新增客户数为：7043


下面统计: 2023-11 的新增情况...
2023-11 的历史年月为：['2023-09' '2023-10']
相较于历史年月，该月的新增客户数为：4732


下面统计: 2023-12 的新增情况...
2023-12 的历史年月为：['2023-09' '2023-10' '2023-11']
相较于历史年月，该月的新增客户数为：4979


下面统计: 2024-01 的新增情况...
2024-01 的历史年月为：['2023-09' '2023-10' '2023-11' '2023-12']
相较于历史年月，该月的新增客户数为：5110




## 加入小循环

In [28]:
for i in range(0, len(month_list)-1):
                   # len()-1 的原因：最后一个月之后就没有数据了
    # 筛选出 month_list 中的每月消费，并统计客户数量
    print(f'下面统计: {month_list[i]} 的新增情况...')
    current_data = df[ df['付款年月']==month_list[i] ]
    current_clients = current_data['脱敏客户ID'].unique()
    
    # =========================== 统计新增情况 ==================================
    # 跳过数据集中的第一个月，因为没有历史数据来验证该客户是否为新增客户
    if i == 0:
        print(f'{month_list[i]} 是第一个月，无需验证客户是否为新增客户。')
        new_clients_num = len(current_clients)
        print(f'该月的新增用户数为：{new_clients_num}')
    else:
        # 筛选该月（current_month）之前的所有历史消费记录
        history_month = month_list[:i]
        print(f'{month_list[i]} 的历史年月为：{history_month}')
        history_data = df[ df['付款年月'].isin(history_month) ]
        # 筛选未在历史消费记录中出现过的新增客户
        new_users = np.setdiff1d(current_data['脱敏客户ID'], history_data['脱敏客户ID'])
        print(f'相较于历史年月，该月的新增客户数为：{len(new_users)}')
        
    # =========================== 统计留存情况 ==================================
    print('-'*50)
    print('下面统计该月之后的每个月的留存情况...')
    for j in range(i+1, len(month_list)):
        next_month_data = df[ df['付款年月']==month_list[j] ]
        # 统计既出现在该月，又出现在下个月的用户
        next_month_retain = np.intersect1d(current_data['脱敏客户ID'], next_month_data['脱敏客户ID'])
        print(f'{month_list[j]} 的留存人数：{len(next_month_retain)}')
    print('\n')

下面统计: 2023-09 的新增情况...
2023-09 是第一个月，无需验证客户是否为新增客户。
该月的新增用户数为：2031
--------------------------------------------------
下面统计该月之后的每个月的留存情况...
2023-10 的留存人数：252
2023-11 的留存人数：216
2023-12 的留存人数：163
2024-01 的留存人数：156
2024-02 的留存人数：164


下面统计: 2023-10 的新增情况...
2023-10 的历史年月为：['2023-09']
相较于历史年月，该月的新增客户数为：7043
--------------------------------------------------
下面统计该月之后的每个月的留存情况...
2023-11 的留存人数：623
2023-12 的留存人数：491
2024-01 的留存人数：488
2024-02 的留存人数：491


下面统计: 2023-11 的新增情况...
2023-11 的历史年月为：['2023-09' '2023-10']
相较于历史年月，该月的新增客户数为：4732
--------------------------------------------------
下面统计该月之后的每个月的留存情况...
2023-12 的留存人数：637
2024-01 的留存人数：562
2024-02 的留存人数：486


下面统计: 2023-12 的新增情况...
2023-12 的历史年月为：['2023-09' '2023-10' '2023-11']
相较于历史年月，该月的新增客户数为：4979
--------------------------------------------------
下面统计该月之后的每个月的留存情况...
2024-01 的留存人数：821
2024-02 的留存人数：636


下面统计: 2024-01 的新增情况...
2024-01 的历史年月为：['2023-09' '2023-10' '2023-11' '2023-12']
相较于历史年月，该月的新增客户数为：5110
---------------------------------

# 在循环中构建 DataFrame 需要的数据

In [42]:
# 存储每月新增的客户数
monthly_increase = {} # 用字典存储，格式输出漂亮些，也有助于添加进 DataFrame 中

for i in range(0, len(month_list)):
    
    # 筛选出 month_list 中的每月消费，并统计客户数量
    print(f'下面统计: {month_list[i]} 的新增情况...')
    current_data = df[ df['付款年月']==month_list[i] ]
    current_clients = current_data['脱敏客户ID'].unique()
    
    # =========================== 统计新增情况 ==================================
    # 跳过数据集中的第一个月，因为没有历史数据来验证该客户是否为新增客户
    if i == 0:
        print(f'{month_list[i]} 是第一个月，无需验证客户是否为新增客户。')
        new_clients_num = len(current_clients)
        print(f'该月的新增用户数为：{new_clients_num}')
        monthly_increase[month_list[i]] = new_clients_num
        
    else:
        # 筛选该月（current_month）之前的所有历史消费记录
        history_month = month_list[:i]
        print(f'{month_list[i]} 的历史年月为：{history_month}')
        history_data = df[ df['付款年月'].isin(history_month) ]
        # 筛选未在历史消费记录中出现过的新增客户
        new_users = np.setdiff1d(current_data['脱敏客户ID'], history_data['脱敏客户ID'])
        print(f'相较于历史年月，该月的新增客户数为：{len(new_users)}')
        monthly_increase[month_list[i]] = len(new_users)
        
    # =========================== 统计留存情况 ==================================
    print('-'*50)
    print('下面统计该月之后的每个月的留存情况...')
    for j in range(i+1, len(month_list)):
        next_month_data = df[ df['付款年月']==month_list[j] ]
        # 统计既出现在该月，又出现在下个月的用户
        next_month_retain = np.intersect1d(current_data['脱敏客户ID'], next_month_data['脱敏客户ID'])
        print(f'{month_list[j]} 的留存人数：{len(next_month_retain)}')
    print('\n')

下面统计: 2023-09 的新增情况...
2023-09 是第一个月，无需验证客户是否为新增客户。
该月的新增用户数为：2031
--------------------------------------------------
下面统计该月之后的每个月的留存情况...
2023-10 的留存人数：252
2023-11 的留存人数：216
2023-12 的留存人数：163
2024-01 的留存人数：156
2024-02 的留存人数：164


下面统计: 2023-10 的新增情况...
2023-10 的历史年月为：['2023-09']
相较于历史年月，该月的新增客户数为：7043
--------------------------------------------------
下面统计该月之后的每个月的留存情况...
2023-11 的留存人数：623
2023-12 的留存人数：491
2024-01 的留存人数：488
2024-02 的留存人数：491


下面统计: 2023-11 的新增情况...
2023-11 的历史年月为：['2023-09' '2023-10']
相较于历史年月，该月的新增客户数为：4732
--------------------------------------------------
下面统计该月之后的每个月的留存情况...
2023-12 的留存人数：637
2024-01 的留存人数：562
2024-02 的留存人数：486


下面统计: 2023-12 的新增情况...
2023-12 的历史年月为：['2023-09' '2023-10' '2023-11']
相较于历史年月，该月的新增客户数为：4979
--------------------------------------------------
下面统计该月之后的每个月的留存情况...
2024-01 的留存人数：821
2024-02 的留存人数：636


下面统计: 2024-01 的新增情况...
2024-01 的历史年月为：['2023-09' '2023-10' '2023-11' '2023-12']
相较于历史年月，该月的新增客户数为：5110
---------------------------------

In [43]:
monthly_increase
# DataFrame 的索引
result_index = list(monthly_increase.keys()) 
# 当月新增
current_month_increase = list(monthly_increase.values())

result_index
current_month_increase

{'2023-09': 2031,
 '2023-10': 7043,
 '2023-11': 4732,
 '2023-12': 4979,
 '2024-01': 5110,
 '2024-02': 7101}

['2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02']

[2031, 7043, 4732, 4979, 5110, 7101]

In [47]:
plus_one = [252, 623, 637, 821, 909, 0]
plus_two = [216, 491, 562, 636, 0, 0]
plus_three = [163, 488, 486, 0, 0, 0]
plus_four = [156, 491, 0, 0, 0, 0]
plus_five = [164, 0, 0, 0, 0, 0]

data = {'当月新增': current_month_increase,
       '+1月': plus_one, '+2月': plus_two, 
       '+3月': plus_three, '+4月': plus_four, '+5月': plus_five}

result = pd.DataFrame(data, index=result_index, columns=list(data.keys()))
result

,当月新增,+1月,+2月,+3月,+4月,+5月
2023-09,2031,252,216,163,156,164
2023-10,7043,623,491,488,491,0
2023-11,4732,637,562,486,0,0
2023-12,4979,821,636,0,0,0
2024-01,5110,909,0,0,0,0
2024-02,7101,0,0,0,0,0


# 留存率表计算

In [53]:
# 求比率
# iloc：行的全部，列的第二行到最后一行
# axis=0: 对横行的 +1~+5月的留存客户数都进行除以当月新增的操作
final = result.divide(result['当月新增'], axis=0).iloc[:, 1:]
final['当月新增'] = result['当月新增']

# 调整列顺序
final = final[['当月新增', '+1月', '+2月', '+3月', '+4月', '+5月']]
final[['+1月', '+2月', '+3月', '+4月', '+5月']] = final[['+1月', '+2月', '+3月', '+4月', '+5月']].applymap(lambda x: str(round(x*100))+'%' )

In [56]:
final.replace('0%', '-', inplace=True)

In [57]:
final

,当月新增,+1月,+2月,+3月,+4月,+5月
2023-09,2031,12%,11%,8%,8%,8%
2023-10,7043,9%,7%,7%,7%,-
2023-11,4732,13%,12%,10%,-,-
2023-12,4979,16%,13%,-,-,-
2024-01,5110,18%,-,-,-,-
2024-02,7101,-,-,-,-,-


In [59]:
final.to_clipboard()

In [60]:
final.to_csv('留存表.csv', index=False)